In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List, Optional
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
#import asyncio
import os
from typing import Tuple, List, Optional

import requests

from pydantic import BaseModel, ValidationInfo, model_validator

import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from src.bubble import *
from src.models import *
from src.utilities import *

Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Expérience utilisateur de Darty : 1710335239022x546888753774592000


In [2]:
types_df = get("Type", constraints=[])
categories_df = get("Category")
subcategories_df = get("SubCategory")
aspects_df = get("Aspect")

In [3]:
aspects_df.to_csv("aspects_df.csv")

In [4]:
ID_CATEG_NONE = categories_df[categories_df["Name"].isna()].iloc[0]["_id"]
REAL_CATEGORIES = categories_df[categories_df["_id"] != ID_CATEG_NONE]
ID_SUBCATEGS_NONE = list(subcategories_df[subcategories_df["Name"].isna()]["_id"])


In [5]:
import pandas as pd


def analyze_feedback_by_period(df, company, category, subcategory, period):
  """Analyzes customer feedback by company, category, subcategory, and period.

  Args:
      df: A pandas DataFrame containing the customer feedback data.
      company: The company to filter by.
      category: The category to filter by.
      subcategory: The subcategory to filter by.
      period: The desired period for grouping the data (e.g., "D" for day, "W" for week, "M" for month, "Y" for year).

  Returns:
      A pandas DataFrame with the following columns:
          * Date: The index of the DataFrame, containing the dates formatted according to the period, 
                  starting from the minimum date of the entire DataFrame.
          * count: The number of customer feedback entries for each period (0 if no entries).
          * ... (similar columns for rating counts, mean, and standard deviation)
          
      The min and std columns will have None values for periods with no entries.
  """

  # Get the minimum date from the entire DataFrame
  min_date = df["Date"].min()

  # Filter the DataFrame by the specified criteria

  filter = (df["Company"] == company)
  if category != ID_CATEG_NONE:
    filter &= (df["Category"] == category)
    if subcategory not in ID_SUBCATEGS_NONE:
      filter &= (df["SubCategory"] == subcategory)

  filtered_df = df[filter]

  # Handle cases where no data matches the filter
  if filtered_df.empty:
    res_df = pd.DataFrame({
        "Date": pd.date_range(start=min_date, periods=0, freq=period)})
    res_df["count"]= 0
    res_df["rating_1"]= 0
    res_df["rating_2"]= 0
    res_df["rating_3"]= 0
    res_df["rating_4"]= 0
    res_df["rating_5"]= 0
    res_df["min"]= 0
    res_df["max"]= 0
    res_df["mean"]= 0
    res_df["std"]= 0
    res_df["median"]= 0
    res_df["q1"]= 0
    res_df["q3"]= 0
    res_df["Date Display"] = res_df["Date"].apply(
      lambda x: format_date(x, period)
  )
    return res_df
  
  # Convert the "Date" column to datetime format
  filtered_df["Date"] = pd.to_datetime(filtered_df["Date"])

  # Define date format function based on period
  def format_date(date, period):
    if period == "D":
      return date.strftime("%d/%m/%Y")
    elif period == "W":
      return date.strftime("%U-%Y")  # Week format with year, e.g., 01-2024
    elif period == "M":
      return date.strftime("%m/%Y")
    else:
      return date.strftime("%Y")

  # Resample the data by the specified period, aggregating on various columns
  grouped_data = (
      filtered_df.set_index("Date")
      .resample(period)
      .agg(
          count=("Rating", "count"),
          rating_1=("Rating", lambda x: x[x == 1].count()),
          rating_2=("Rating", lambda x: x[x == 2].count()),
          rating_3=("Rating", lambda x: x[x == 3].count()),
          rating_4=("Rating", lambda x: x[x == 4].count()),
          rating_5=("Rating", lambda x: x[x == 5].count()),
          mean=("Rating", "mean"),
          std=("Rating", "std"),
          median=("Rating", "median"),
          min=("Rating", "min"),
          max=("Rating", "max"),
          q1=("Rating", lambda x: x.quantile(0.25)),
          q3=("Rating", lambda x: x.quantile(0.75)),
      )
      .reset_index()
  )

  
  # Set the minimum date from entire DataFrame as the starting index
  grouped_data.set_index("Date", inplace=True)
  # Create a date range with appropriate length based on min and max dates of filtered data
  date_range = pd.date_range(
      start=min_date, end=filtered_df["Date"].max(), freq=period
  )

  # Reindex to fill missing periods with 0 and None for mean/std
  grouped_data = grouped_data.reindex(date_range).fillna(0)

  # Fill missing values with appropriate values
  grouped_data.loc[:, "count"] = grouped_data["count"].fillna(0)
  grouped_data.loc[grouped_data["count"] == 0, ["mean", "std", "median", "min", "max", "q1", "q3"]] = None

  grouped_data = grouped_data.rename_axis('Date').reset_index()

  # Apply date formatting based on period
  grouped_data["Date Display"] = grouped_data["Date"].apply(
      lambda x: format_date(x, period)
  )
  return grouped_data


In [6]:
def to_list(s):
    return [0 if pd.isna(x) else x for x in s]

def send_aspects(df, company, category, subcategory, period, grouped_by=None):
    d = {
        "Company": company,
        "Project": PROJECT_ID,
        "Category": category,
        "SubCategory": subcategory,
        "Period": period,
        "Grouped by": grouped_by,
        "Counts": to_list(df["count"]),
        "Counts of 1s": to_list(df["rating_1"]),
        "Counts of 2s": to_list(df["rating_2"]),
        "Counts of 3s": to_list(df["rating_3"]),
        "Counts of 4s": to_list(df["rating_4"]),
        "Counts of 5s": to_list(df["rating_5"]),
        "Dates": to_list(df["Date"].apply(lambda x:pd.to_datetime(x).strftime("%m/%d/%Y"))), #
        "Dates Display": to_list(df["Date Display"]),
        "Max Ratings": to_list(df["min"]),
        "Mean Ratings": to_list(df["mean"]),
        "Std Ratings": to_list(df["std"]),
        "Med Ratings": to_list(df["median"]),
        "Min Ratings": to_list(df["max"]),
        "Q1 Rating": to_list(df["q1"]),
        "Q3 Rating": to_list(df["q3"]),
        }  

    bubble_id = bubble_client.create("Aspect Evol",d)
    return bubble_id

In [7]:
aspects_df

,Company,Project,Rating,SubCategory,Associated_feedback,Date,Category,_id,Explanation
0,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710338634358x749419887693214300,1710335410193x146397114086602720,2024-02-07 00:00:00+00:00,1710338633710x927942518793102100,1710344634381x725785970497039500,NaN
1,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710338628331x841166890947339300,1710335410193x146397114086602720,2024-02-07 00:00:00+00:00,1710338627647x997830309269291800,1710344634385x589643706464383700,Le clavier Magic Keyboard avec Touch ID et pav...
2,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710338629441x225256799042269150,1710335410195x175970366374120320,2024-01-31 00:00:00+00:00,1710338627647x997830309269291800,1710344634394x626110569216282900,Appréciation pour l'installation de la machine...
3,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710338624941x617116569147634600,1710335410202x536776314861663740,2024-01-05 00:00:00+00:00,1710338624344x162302040876331800,1710344634399x191041472624027260,Livraison de la machine à laver Bosch jugée im...
4,1707313014508x102198350946437700,1710335239022x546888753774592000,1,1710338632433x109511910948359100,1710335410203x151859605597644900,2023-11-08 00:00:00+00:00,1710338630923x768035961851395700,1710344634405x825865459861836000,Refus de remboursement pour un pack Office ach...
5,1707313014508x102198350946437700,1710335239022x546888753774592000,1,1710338631676x473260343218697300,1710335410205x126748698162511440,2023-10-06 00:00:00+00:00,1710338630923x768035961851395700,1710344634408x371921425465531400,Expérience négative avec une vendeuse jugée in...
6,1707313014508x102198350946437700,1710335239022x546888753774592000,1,1710338624941x617116569147634600,1710335410207x309366621526354000,2024-02-07 00:00:00+00:00,1710338624344x162302040876331800,1710344634411x912620933734360400,Changement non respecté pour la date de livrai...
7,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710338635064x919561724451331200,1710335410209x922875138708661500,2023-06-01 00:00:00+00:00,1710338633710x927942518793102100,1710344634414x244446453514186140,NaN
8,1707313014508x102198350946437700,1710335239022x546888753774592000,1,1710338629441x225256799042269150,1710335410210x468128101846428900,2024-02-05 00:00:00+00:00,1710338627647x997830309269291800,1710344634416x657624230504599700,Problèmes de branchement par le livreur et abs...
9,1707313014508x102198350946437700,1710335239022x546888753774592000,5,1710338634358x749419887693214300,1710335410211x593588402850767400,2024-02-02 00:00:00+00:00,1710338633710x927942518793102100,1710344634419x912127677738184000,NaN


In [8]:
subcategories_df

,Company,Name,Project,Category,_id
0,1707313014508x102198350946437700,Navigation sur le site,1710335239022x546888753774592000,1710338620323x620032078794710700,1710338620651x814545058902816000
1,1707313014508x102198350946437700,Processus de commande,1710335239022x546888753774592000,1710338620323x620032078794710700,1710338621053x133318659995601550
2,1707313014508x102198350946437700,Options de paiement,1710335239022x546888753774592000,1710338620323x620032078794710700,1710338622145x749606343311301400
3,1707313014508x102198350946437700,Disponibilité des produits,1710335239022x546888753774592000,1710338620323x620032078794710700,1710338622788x634503680852456700
4,1707313014508x102198350946437700,Promotions et offres,1710335239022x546888753774592000,1710338620323x620032078794710700,1710338623186x688630926001471600
5,1707313014508x102198350946437700,Rapport qualité-prix,1710335239022x546888753774592000,1710338620323x620032078794710700,1710338623651x560598844301543000
6,1707313014508x102198350946437700,NaN,1710335239022x546888753774592000,1710338620323x620032078794710700,1710338624047x387942420416467840
7,1707313014508x102198350946437700,Délais de livraison,1710335239022x546888753774592000,1710338624344x162302040876331800,1710338624941x617116569147634600
8,1707313014508x102198350946437700,Conditionnement et emballage,1710335239022x546888753774592000,1710338624344x162302040876331800,1710338625823x991173288031338200
9,1707313014508x102198350946437700,Suivi de la commande,1710335239022x546888753774592000,1710338624344x162302040876331800,1710338626208x286957968379819800


In [9]:
for category in tqdm(categories_df["_id"]):
    subcategories = subcategories_df[subcategories_df["Category"]==category]["_id"]

    for subcategory in subcategories:
        for period in ["W", "Y"]: #["D","W", "M", "Y"]:
            df = analyze_feedback_by_period(aspects_df, COMPANY_ID, category, subcategory, period)
            send_aspects(df, COMPANY_ID, category, subcategory, period)

  0%|          | 0/8 [00:00<?, ?it/s]

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_53270/2947707909.py:40: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  "Date": pd.date_range(start=min_date, periods=0, freq=period)})
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_53270/2947707909.py:40: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  "Date": pd.date_range(start=min_date, periods=0, freq=period)})
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_53270/2947707909.py:40: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  "Date": pd.date_range(start=min_date, periods=0, freq=period)})
/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_53270/2947707909.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [11]:
df[[f"Count of {i}s" for i in range(1,6)]].plot(kind='bar', stacked=True)


KeyError: "None of [Index(['Count of 1s', 'Count of 2s', 'Count of 3s', 'Count of 4s',\n       'Count of 5s'],\n      dtype='object')] are in the [columns]"